In [1]:
import requests
import openai
from openai import OpenAI
from dotenv import load_dotenv
import os
import zipfile
from tqdm import tqdm
import base64
import pathlib

In [2]:
load_dotenv()

True

In [3]:
client = OpenAI()

In [4]:
texts = {}
facts = []

# Ścieżka do folderów
reports_folder = 'materials'
facts_folder = 'materials/facts'

# Załaduj pliki raportów
for filename in os.listdir(reports_folder):
    if filename.endswith('.txt'):
        with open(os.path.join(reports_folder, filename), 'r', encoding='utf-8') as f:
            texts[filename] = f.read()

# Załaduj fakty
for filename in os.listdir(facts_folder):
    if filename.endswith('.txt'):
        with open(os.path.join(facts_folder, filename), 'r', encoding='utf-8') as f:
            facts.append(f.read())

# Połącz wszystkie fakty w jedną zmienną tekstową
facts_context = "\n".join(facts)

In [56]:
# Usuń wszystkie wiersze zawierające "entry deleted"
facts_filtered = [fact for fact in facts if "entry deleted" not in fact]

# Połącz fakty w jeden tekst, po usunięciu niechcianych wierszy
facts_context = "\n".join(facts_filtered)

In [146]:
def ask_text(report_text, facts_text):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": f'Treść raportu: {report_text}'},
            # {"role": "system", "content": f'Oto dodatkowe fakty, które mogą pomóc w zrozumieniu kontekstu: {facts_text}'},
            {"role": "user", "content": "Wygeneruj słowa kluczowe w formie mianownika. \
            Oddziel słowa przecinkami. Nie pomijaj żadnych istotnych informacji, osób, nauczycieli czy zwierząt, ani info o odciskach Zawadzkiej, gdzie zostały odnalezione.\
            Przy generowaniu metadanych posiłkuj się całą posiadaną wiedzą wraz z dodatkowymi faktami."}
        ]
    )
    answer = response.choices[0].message.content
    return answer

In [147]:
keywords = {}
for filename, text in texts.items():
    keywords[filename] = ask_text(text, facts_context)

In [149]:
keywords

{'2024-11-12_report-00-sektor_C4.txt': 'Godzina, jednostka, północne skrzydło, fabryka, Aleksander Ragowski, skan biometryczny, baza danych, dział kontroli, patrol.',
 '2024-11-12_report-01-sektor_A1.txt': 'Godzina, alarm, wykrycie, ruch, organizm, analiza, wizualizacja, sensoryka, obecność, zwierzyna, leśna, fałszywy alarm, obszar, bezpieczeństwo, trasa, patrol, spokój.',
 '2024-11-12_report-02-sektor_A3.txt': 'godzina, obszar, patrol, noc, aktywność, organiczność, mechanika, monitoring, peryferia, obiekt, zadania, kontynuacja',
 '2024-11-12_report-03-sektor_A3.txt': 'Godzina, Patrole, Noc, Poziom, Czujniki, Wytyczne, Wykrywanie, Życie organiczne, Rezultaty, Stan, Zakłócenia',
 '2024-11-12_report-04-sektor_B2.txt': 'godzina, patrol, zachód, teren, anomalie, odchylenia, norma, sektor, bezpieczeństwo, kanały komunikacyjne, punkt, Zawadzka, odciski',
 '2024-11-12_report-05-sektor_C1.txt': 'godzina, aktywność, organiczna, technologiczna, sensor, dźwięk, detektor, ruch, gotowość, sygnał, p

In [179]:
solution_response = {
   "task": "dokumenty",
   "apikey": os.getenv('AI_DEVS_3_API_KEY'),
   "answer": {
  "2024-11-12_report-00-sektor_C4.txt": "jednostka organiczna, Aleksander Ragowski, nauczyciel, skan biometryczny, północne skrzydło fabryki, dział kontroli, opozycja",
  "2024-11-12_report-01-sektor_A1.txt": "ruch organiczny, alarm fałszywy, lokalna zwierzyna leśna, sektor A1, bezpieczeństwo",
  "2024-11-12_report-02-sektor_A3.txt": "monitoring, brak aktywności, obszar patrolowy, nocny patrol, sektor A3, peryferie",
  "2024-11-12_report-03-sektor_A3.txt": "nocne patrole, brak życia organicznego, czujniki aktywne, monitoring, cisza, sektor A3",
  "2024-11-12_report-04-sektor_B2.txt": "zachodni teren, brak anomalii, komunikacja, sektor B2, bezpieczeństwo",
  "2024-11-12_report-05-sektor_C1.txt": "czujniki ruchu, brak aktywności, monitoring dźwiękowy, sektor C1, patrol",
  "2024-11-12_report-06-sektor_C2.txt": "sektor północno-zachodni, skanery temperatury, brak wykrycia, patrol, sektor C2",
  "2024-11-12_report-07-sektor_C4.txt": "ultradźwiękowy sygnał, nadajnik, Barbara Zawadzka, odciski palców, sektor C4, JavaScript, frontend, programista, ruch oporu, dział śledczy, zabezpieczenie",
  "2024-11-12_report-08-sektor_A1.txt": "monitoring, brak ruchu, sektor A1, cisza, brak aktywności",
  "2024-11-12_report-09-sektor_C2.txt": "peryferie zachodnie, czujniki, brak sygnałów, sektor C2, zakończenie cyklu, bezpieczeństwo"
}

}

In [180]:
response = requests.post('https://centrala.ag3nts.org/report', json=solution_response)

In [181]:
response

<Response [200]>

In [182]:
response.json()

{'code': 0, 'message': '{{FLG:NOWYOUKNOW}}'}